# Project 2 - Binary Classification Comparative Methods

For this project we're going to attempt a binary classification of a dataset using multiple methods and compare results.

Our goals for this project will be to introduce you to several of the most common classification techniques, how to perform them and tweek parameters to optimize outcomes, how to produce and interpret results, and compare performance. You will be asked to analyze your findings and provide explanations for observed performance.


<b><u>DEFINITIONS</b></u>


<b> Binary Classification:</b>
In this case a complex dataset has an added 'target' label with one of two options. Your learning algorithm will try to assign one of these labels to the data.

<b> Supervised Learning:</b>
This data is fully supervised, which means it's been fully labeled and we can trust the veracity of the labeling.

In [ ]:
#Here are a set of libraries we imported to complete this assignment.
#Feel free to use these or equivalent libraries for your implementation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph
import matplotlib
import os
import time
#Sklearn classes
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn import metrics
from sklearn.svm import SVC  #SVM classifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
import sklearn.metrics.cluster as smc

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer

from matplotlib import pyplot as plt
import itertools

%matplotlib inline

#Sets random seed
import random
random.seed(42)

#Mouting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# import the provided helper functions
from helper import save_fig, draw_confusion_matrix, heatmap, make_meshgrid, plot_contours

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-82a6bb54ece1>", line 36, in <cell line: 36>
    get_ipython().run_line_magic('cd', "'/content/drive/MyDrive/CM148_Project2'")
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the ab

In [16]:
!pwd

/root


In [3]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


# Example Project

In this part, we will go over how to perform a Binary classification task using a variety of models. We will provide examples of how to train and evaluate these models.

## Dataset Description

Healthcare is an important industry that uses machine learning to aid doctors in diagnosing many different kinds of illnesses and diseases. For this example project, we will be using the [Breast Cancer Wisconsin Dataset](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data) to determine whether a mass found in a body is benign or malignant.

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image.

Feature Information:

Column 1: ID number

Column 2: Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus:

    1. radius (mean of distances from center to points on the perimeter)
    2. texture (standard deviation of gray-scale values)
    3. perimeter
    4. area
    5. smoothness (local variation in radius lengths)
    6. compactness (perimeter^2 / area - 1.0)
    7. concavity (severity of concave portions of the contour)
    8. concave points (number of concave portions of the contour)
    9. symmetry
    10. fractal dimension ("coastline approximation" - 1)

Due to the statistical nature of the test, we are not able to get exact measurements of the previous values. Instead, the dataset contains the mean and standard error of the real-valued features.

Columns 3-12 present the mean of the measured values

Columns 13-22 present the standard error of the measured values

## Load and Analyze the dataset

Mounted at /content/drive


In [ ]:
#Load Data
data = pd.read_csv(/content/sample_data)

Always look at your dataset after loading it. Use information from .describe and .info to learn more about the dataset.

In [ ]:
data.head(5)

In [ ]:
data.describe()

In [ ]:
data.info()

While .info shows that every entry has 569 non-null and there are 569 entries, it is good to explicitly check for nulls.

In [ ]:
data.isnull().sum()

Awesome! No need for imputation!

While we are looking at the dataset, we shall remove the "id" column.

In [ ]:
data = data.drop(["id"],axis= 1)

### Looking at the target labels

For this project, we wish to classify the diagnosis column.

In [ ]:
data["diagnosis"]

We need to transform this column into numerical column so that we may use them in our models. To do this, we will employ the LabelEncoder to automatically transform all the target label.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['diagnosis'] = le.fit_transform(data['diagnosis'])
print(le.classes_)

NameError: name 'data' is not defined

In [ ]:
data['diagnosis']

### Let's look at a histogram of the full dataset.

Its always good to get a global view of your datasets by looking at their histograms. You might see some interesting trends.

In [ ]:
data.hist(figsize = (20,15))
plt.show()

From the histograms, we can see some interesting trends. Possible observations:

- Many of the _se columns indicate a heavy skewness towards low values and have large tails
- Many of the _mean columns look more Gaussian in shape
- There is a large disparity between the ranges of certain features. For example, radius mean can go from 0 to 25 while smoothness_mean is in the range [0.050,0.150]. This indicates we will have to normalize or standardize the features if the models are sensitive to such measures.

### Looking at the correlation matrix to get an idea about which features are important

In [ ]:
correlations = data.corr()
columns = list(data)
# Creates the heatmap
heatmap(correlations.values, columns, columns, figsize=(20, 12), cmap="hsv")

In [ ]:
# Let's specifically look at the correlations of our target feature
correlations["diagnosis"].sort_values(ascending=False)

We can see that there is a lot of correlation between the features and the target label. Thus, we can expect to learn something from the data

### When doing classification, check if classes are heavily imbalanced.

It is important that the dataset does not prefer one class over any others. Otherwise, it may bias the model to not learn the minority classes well.

Lets use a histogram and count the number of elements in each class.

In [ ]:
data['diagnosis'].hist(bins=2, figsize=(5,5))
data['diagnosis'].value_counts()

There is a bit of an imbalance which is something to keep in mind if we find that our models do not perform well on the minority classes. For our purposes, this imbalance is not big enough to be an issue so we will not perform balancing techniques for this dataset.

Since the dataset is small though, we want to be careful when making training and testing splits to ensure that there is enough of each class for both splits. We will show how to perform this shortly.

### Setting up the data

Before starting any model training, we have to split up the target labels from our features.

In [ ]:
y = data["diagnosis"]
x = data.drop(["diagnosis"],axis = 1)

Now, we also split the data into training and testing data. To ensure that there is not an imbalance of classes in the training and testing set, we will use the stratify parameter in train_test_split to perform stratified sampling on the data (Recall from lecture how stratified sampling is performed).

In [ ]:
train_raw, test_raw, target, target_test = train_test_split(x,y, test_size=0.2, stratify= y, random_state=0)

Note that we marked the input feature data as raw to indicate that there has been no pre-processing on them such as standardization. Shortly, we will show the affect that pre-processing has on the performance of the model.

Let us quickly test that the splits are somewhat balanced.

In [ ]:
#Training classes
target.hist(bins=2, figsize=(5,5))
target.value_counts()

In [ ]:
#Testing classes
target_test.hist(bins=2, figsize=(5,5))
target_test.value_counts()

We can see that the class balance is about the same as before the split. In fact, we can see that if a classifier just guessed class 0, it would have an accuracy of $100 \times \frac{72}{72+42} = 63.15\%$. We can consider this the minimum accuracy for your model to compare against.

## Models for Classification: KNN

For our first model, we will use KNN classfication. This is a model we have seen many times throughout the course and it would be interesting to see how well it performs.

### Simple KNN classification with K = 3

Let us try KNN on the raw data with simply 3 nearest neighbors. We use the sklearn [metric library](https://scikit-learn.org/stable/modules/model_evaluation.html) to calculate the measures of interest. In this case, we focus on accuracy.

In [ ]:
# k-Nearest Neighbors algorithm
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_raw, target)
predicted = knn.predict(test_raw)

In [ ]:
print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))

We can see that there is already a huge improvement in accuracy on comparison to the baseline of 63.15%. Let's see the effect that standardizing the input features would have on the KNN performance.

### Affect of pre-processing on KNN

In [ ]:
# Since all features are real-valued, we only have one pipeline
pipeline = Pipeline([("scaler", StandardScaler())])

# Transform raw data. Note that we don't want to fit the test data
train = pipeline.fit_transform(train_raw)
test = pipeline.transform(test_raw)

In [ ]:
# k-Nearest Neighbors algorithm
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train, target)
testing_result = knn.predict(test)
predicted = knn.predict(test)

In [ ]:
print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))

We can see that with pre-processing we were able to get a much better classification accuracy.

Here we only used StandardScaler. Lets see if other pre-processing techniques could have also worked. As such, lets  look at MinMaxScaler and Normalizer:

In [ ]:
preprocessors = [StandardScaler(),MinMaxScaler(),Normalizer() ]

for pre in preprocessors:
    pipeline = Pipeline([
        ('preprocessor', pre)
    ])


    #Transform raw data
    train = pipeline.fit_transform(train_raw)
    test = pipeline.transform(test_raw) #Note that there is no fit calls
    # k-Nearest Neighbors algorithm
    knn = KNeighborsClassifier(n_neighbors=7)
    knn.fit(train, target)
    testing_result = knn.predict(test)
    predicted = knn.predict(test)

    print(pre)
    print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))

We can see that MinMaxScaler had the same performance as StandardScaler. Yet, Normalizer did not improve the model.

### Visualizing decision boundaries  for KNN

Its always nice to see the decision boundaries a model decides upon. Let's see how the decision boundary changes as function of k when only using the two most correlated features to the target labels: concave_points_mean and perimeter_mean.

In [ ]:
# Extract first two features and use the standardscaler
train_2 = StandardScaler().fit_transform(
    train_raw[["concave points_mean", "perimeter_mean"]]
)

k_r = [1, 3, 5, 7]
for k in k_r:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_2, target)
    draw_contour(train_2, target, knn, class_labels=["Benign", "Malignant"])
    plt.title(f"K ={k}")

We can see that as k gets larger, the decision boundary gets smoother.

## Models for Classification: Logistic Regression

While KNN is a very powerful model, it does come with a few issues such as

- Require storing the full training dataset
- Prediction is done by comparing new sample will all samples in training set which is time-consuming

These issues arise because KNN is a **non-parametric** model which means that it does not summarize the data into a finite set of parameters.

Let us now look at Logistic Regression which is an example of a **parametric** model.

### Simple Logistic Regression


First, let us see how logistic regression performs without any regularization.

In [ ]:
log_reg = LogisticRegression(penalty="l2", max_iter=10000, solver="lbfgs", C=10**30)
# C is choosen to be high to remove regularization
# We could have chosen penalty = "none" since lbfgs supports it but this option is not possible for all solvers.

log_reg.fit(train_raw, target)
testing_result = log_reg.predict(test_raw)
predicted = log_reg.predict(test_raw)

print("%-12s %f" % ("Accuracy:", metrics.accuracy_score(target_test, predicted)))

We can see that Logistic Regression is actually performing much better than any of the KNN models we tried. We can also see the parameters that the model learned.

In [ ]:
#Parameters for each feature
log_reg.coef_

In [ ]:
#Intercept term
log_reg.intercept_

In [ ]:
print("Number of Features in data:", train_raw.shape[1])
print("Number of Parameters:", len(log_reg.coef_[0]))

Since we are using Logistic Regression where we model the log odds with a linear function, it makes sense that we have a parameter/coefficient for each input feature.

###  Parameters for Logistic Regression

In Sci-kit Learn, the following are just some of the parameters we can pass into Logistic Regression:

- penalty: {'l1', 'l2', 'elasticnet’, 'none’} default="l2"
    - Specifies the type of regularization to use. Not all penalties work for each solver.
- C: positive float, default=1
    - Inverse of the regularization strength. You can treat C as $\frac{1}{\lambda}$ as shown in lecture. Thus, as C gets smaller, the regularization strength increases.
- solver: {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, default=’lbfgs’
    - Algorithm to use in the optimization problem. Each algorithms solves logistic regression using different iterative methods that are based on the gradient. Read the [sci-kit learn documentation](https://scikit-learn.org/dev/modules/linear_model.html#logistic-regression) for more information.
- max_iter: int, default=100
    - Maximum number of iterations taken for the solvers to converge.

Each parameter has a different effect on the model. Let's look at how the choose of max_iter affects the model performance on the raw data and the standardized dataset.

In [ ]:
#Since all features are real-valued, we only have one pipeline
preprocesser = Pipeline([
    ('scaler', StandardScaler())
])


#Transform raw data
train = preprocesser.fit_transform(train_raw)
test = preprocesser.transform(test_raw) #Note that there is no fit call

In [ ]:
log_reg = LogisticRegression(penalty = "l2",max_iter = 2000, solver = "lbfgs", C= 0.01)

#Train raw is the data before preprocessing
log_reg.fit(train_raw, target)
predicted = log_reg.predict(test_raw)
print("%-12s %f" % ('Raw Data Accuracy:', metrics.accuracy_score(target_test,predicted)))


#Train is the data after preprocessing (using Standard scalar)
log_reg.fit(train, target)
predicted = log_reg.predict(test)

print("%-12s %f" % ('Preprocessed Data Accuracy:', metrics.accuracy_score(target_test,predicted)))

We see that the accuraccies are pretty close to each other.  Lets see what happens when we decrease the max_iter.

In [ ]:
log_reg = LogisticRegression(penalty="l2", max_iter=70, solver="lbfgs", C=0.01)

# Train raw is the data before preprocessing
log_reg.fit(train_raw, target)
predicted = log_reg.predict(test_raw)
print(
    "%-12s %f" % ("Raw Data Accuracy:", metrics.accuracy_score(target_test, predicted))
)

In [ ]:
# Train is the data after preprocessing (using Standard scalar)
log_reg.fit(train, target)
predicted = log_reg.predict(test)

print(
    "%-12s %f"
    % ("Preprocessed Data Accuracy:", metrics.accuracy_score(target_test, predicted))
)

Ooops! The model did not seem to converge. Its seem that the scale of the features strongly affects the convergence speed of the iterative algorithm. As suggested, we can fix this issue by increaing the max_iter, re-scaling the data, or using a different solver.


### Cross Validation for Logistic Regression

Let us do a little experiment using cross validation to see how each term affects the logistic regression. We will perform this example on the standardized data.

In [ ]:
# You may even do Cross validation for classification

from sklearn.model_selection import GridSearchCV

# Note that this a list of dict
# Each dict describes the combination of parameters to check
parameters = [
    {
        "penalty": ["l2"],
        "C": [0.01, 1, 100],
        "solver": ["lbfgs", "liblinear"],
    },  # These solvers support penalty = "l2"
    {
        "penalty": [None],
        "C": [1],  # Specified to prevent error message
        "solver": ["lbfgs", "newton-cg"],  # These solvers support penalty = "none"
    },
]

# instantiate model

# Implementing cross validation
k = 3
kf = KFold(n_splits=k, random_state=None)

# will change parameters during CV
log_reg = LogisticRegression(penalty="none", max_iter=1000, solver="lbfgs")
grid = GridSearchCV(log_reg, parameters, cv=kf, scoring="accuracy")
grid.fit(train, target)

In [ ]:
#Put results into Dataframe
res= pd.DataFrame(grid.cv_results_)
res

In [ ]:
# Extract the columns that specify the score and the parameters for each row
res[["rank_test_score", "param_C", "param_penalty", "param_solver", "mean_test_score"]]

We can see that the choice of these parameters can stronlgy affect performance of the classifier. Lets check the performance of the best parameters on the test set.

In [ ]:
#Train raw is the data before preprocessing
predicted = grid.predict(test)
print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))

Note that this test accuracy is not as good as some of the other logistic regression examples we've shown.

### Speedtest between KNN and Logistic Regression

Lets see how long KNN and Logistic Regression take to perform training and testing.


In [ ]:
scaler = StandardScaler()
train = scaler.fit_transform(train_raw)
test = scaler.fit_transform(test_raw)


log_reg = LogisticRegression(penalty=None, max_iter=1000)
knn = KNeighborsClassifier(n_neighbors=3)

t0 = time.time()
knn.fit(train, target)
t1 = time.time()
print("KNN Training Time : ", t1 - t0)

t0 = time.time()
log_reg.fit(train, target)
t1 = time.time()
print("Logistic Regression Training Time : ", t1 - t0)

In [ ]:
t0 = time.time()
knn.predict(test)
t1 = time.time()
print("KNN Testing Time : ", t1-t0)

t0 = time.time()
log_reg.predict(test)
t1 = time.time()
print("Logistic Regression Testing Time : ", t1-t0)

This simple test shows that Logistic Regression is slower than KNN during Training time but is much faster during testing time.

### Visualizing decision boundaries for Logistic Regression

Now, lets look at the decision boundary caused by Logistic Regression. Same as for KNN, we use the two most correlated features to the target labels: concave_points_mean and perimeter_mean. This way, we can visualize the 2D decision boundary.

In [ ]:
#Extract first two feature and use the standardscaler
train_2 = StandardScaler().fit_transform(train_raw[['concave points_mean','perimeter_mean']])

Cs  = [0.001,0.1,1000]
for C in Cs:
    log_reg = LogisticRegression(penalty = "l2",max_iter = 1000, solver = "lbfgs", C=C) #will change parameters during CV
    log_reg.fit(train_2, target)

    draw_contour(train_2,target,log_reg, class_labels = ['Benign', 'Malignant'])
    plt.title(f"C ={C}")

We can see as the regularization strength changes, the decision boundary moves as well. Additionally, we can clearly see that the decision boundary is a line since this is a linear model.

## Models for Classification: SVM

We now discuss another type of linear classification model known as Support Vector Machines (SVM). Where Logistic Regression was motivated probability theory, SVM is motivated by geometeric arguments. Specifcally, SVM finds a separating hyperline that maximizes the margin (i.e. distance from each class). The hyperplane is used to classify the points by designating every sample on of side of the hyperplane as the positive class and the other side as the negative class.

The hyperplane is determine by a few sample points known as support vectors that uniquely characterize the hyperplane.


![svm_im](images/support-vector-machine-algorithm.png)


Note that it may not always be possible to find a hyperplane that completely separates the classes. Thus, we use what is known as Soft-Margin SVM which aims to maximize the margin while minizming the distance on the classes that are on the wrong side.

All Sci-kit learn implementations of SVM that we use are soft-margin SVM.

### Simple SVM classification

In [ ]:
svm = SVC()
svm.fit(train, target)
predicted = svm.predict(test)
print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))

###  Parameters for SVM

In Sci-kit Learn, the following are just some of the parameters we can pass into Logistic Regression:

- C: positive float, default=1
    - Inverse of the regularization strength. You can treat C as $\frac{1}{\lambda}$ as shown in lecture. Thus, as C gets smaller, the regularization strength increases. SVM only uses the L2 regularization.
- kernel: {'linear’, 'poly’, 'rbf’, 'sigmoid’}, default=’rbf’
    - Specifies the kernel type to be used in the algorithm. A kernel specifies a mapping into a higher dimension space to allow for non-linear decision boundaries.
- degree: int, default=3
    - Degree of the polynomial kernel function ('poly’). Ignored by all other kernels.
    

### Visualizing decision boundaries for SVM

Now, lets look at the decision boundary caused by SVM with different kernels. Same as for KNN and Logistic Regression, we use the two most correlated features to the target labels: concave_points_mean and perimeter_mean. This way, we can visualize the 2D decision boundary.


In [ ]:
#Extract first two feature and use the standardscaler
train_2 = StandardScaler().fit_transform(train_raw[['concave points_mean','perimeter_mean']])

kernel  = ['linear', 'poly', 'rbf',  'sigmoid']
for ker in kernel:
    svm = SVC(kernel = ker) #will change parameters during CV
    svm.fit(train_2, target)
    draw_contour(train_2,target,svm,class_labels = ['Benign', 'Malignant'])

    plt.title(f"Kernel ={ker}")

We can see that the decision boundary is not always linear because we are using non-linear kernels.

## Important Measures for Classifications

Now that we have gone over a few models for binary classification, let's explore the different ways we can measure the performance of these models.

In [ ]:
#Example classifier
log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(train_raw, target)
predicted = log_reg.predict(test_raw)

Here are just some of the most important measures of interest. We use the convention to refer to the class labeled as $1$ as the positive class.

- **Accuracy:** The percentage of predictions that are correct. Use metrics.accuracy_score
- **Precision:** $\frac{\text{Number of labels correctly classified as positive}}{\text{Number of labels classified as positives}}$. Percentage of predictions that are correctly positive among all the predictions that were classified as positive. Use metrics.precision_score
- **Recall:** $\frac{\text{Number of labels correctly classified as positive}}{\text{Number of labels where the true class is positive}}$. Percentage of predictions that are correctly positive among all the labels where the true class is positive. Also known as the probability of detecting when a class is positive. Use metrics.recall_score
- **F1 Score:** Harmonic mean of the precision and recall. Highest value is $1$ when both precision and recall are $1$, i.e. perfect. Lowest value is $0$ when either precision or recall is zero. Provides an aggregate score to analyze both precision and recall. Use metrics.f1_score

We can calculate these measures by using a confusion matrix as well.

In [ ]:
print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))
print("%-12s %f" % ('Precision:', metrics.precision_score(target_test,predicted, labels=None, pos_label=1, average='binary', sample_weight=None)))
print("%-12s %f" % ('Recall:', metrics.recall_score(target_test,predicted, labels=None, pos_label=1, average='binary', sample_weight=None)))
print("%-12s %f" % ('F1 Score:', metrics.f1_score(target_test,predicted, labels=None, pos_label=1, average='binary', sample_weight=None)))
print("Confusion Matrix: \n", metrics.confusion_matrix(target_test,predicted))

#Draws confusion matrix
draw_confusion_matrix(target_test, predicted, ['Benign', 'Malignant'])